In [1]:
from os import urandom
from enum import Enum
from pygost.gost3412 import GOST3412Kuznechik
from pygost.mgm import MGM
from pygost.mgm import nonce_prepare
from pygost import gost3410
from pygost import gost34112012256
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256
from gostcrypto import gosthmac


class CipherSuiteEnum(Enum):
    DH_AES_128_GCM_SHA256 = 1
    ECDH_KUZNYECHIK_MGM_STREEBOG = 2


class CipherSuite:
    def encrypt(self, k, m):
        pass
    
    def decrypt(self, k, c, tag, nonce):
        pass
    
    def mac(self, data, key):
        pass
    
    def check_mac(self, mac, data, key):
        pass


class DH_AES_128_GCM_SHA256(CipherSuite):
    def get_enum(self):
        return CipherSuiteEnum.DH_AES_128_GCM_SHA256
    
    def mac(self, data, key):
        h = HMAC.new(key, digestmod=SHA256)
        h.update(data)
        return h.digest()
    
    def check_mac(self, mac, data, key):
        h = HMAC.new(key, digestmod=SHA256)
        h.update(data)
        return h.digest() == mac
    
    def encrypt(self, k, m):
        nonce = urandom(16)
        cipher = AES.new(k, AES.MODE_GCM, nonce)
        ct = cipher.encrypt(m)
        tag = cipher.digest()
        return ct, tag, nonce
        
    def decrypt(self, k, c, tag, nonce):
        cipher = AES.new(k, AES.MODE_GCM, nonce)
        return cipher.decrypt_and_verify(c, tag)


class ECDH_KUZNYECHIK_MGM_STREEBOG(CipherSuite):
    def get_enum(self):
        return CipherSuiteEnum.ECDH_KUZNYECHIK_MGM_STREEBOG
    
    def mac(self, data, key):
        hmac = gosthmac.new('HMAC_GOSTR3411_2012_256', key, data=data)
        return hmac.digest()
    
    def check_mac(self, mac, data, key):
        hmac = gosthmac.new('HMAC_GOSTR3411_2012_256', key, data=data)
        return hmac.digest() == mac
    
    def encrypt(self, k, m):
        mgm = MGM(GOST3412Kuznechik(k).encrypt, GOST3412Kuznechik.blocksize)
        nonce = nonce_prepare(urandom(16))
        seal = mgm.seal(nonce, m, b'')
        ct = seal[:-mgm.tag_size]
        tag = seal[-mgm.tag_size:]
        return ct, tag, nonce
    
    def decrypt(self, k, c, tag, nonce):
        mgm = MGM(GOST3412Kuznechik(k).encrypt, GOST3412Kuznechik.blocksize)
        nonce = nonce_prepare(nonce)
        return mgm.open(nonce, c + tag, b'')

In [2]:
# Tests

key = b'\x11' * 32
message = b'hello'

cipher_suite = DH_AES_128_GCM_SHA256()
ct, tag, nonce = cipher_suite.encrypt(key, message)
print(cipher_suite.decrypt(key, ct, tag, nonce))
mac = cipher_suite.mac(message, key)
print(cipher_suite.check_mac(mac, message, key))
print(cipher_suite.get_enum())

cipher_suite = ECDH_KUZNYECHIK_MGM_STREEBOG()
ct, tag, nonce = cipher_suite.encrypt(key, message)
print(cipher_suite.decrypt(key, ct, tag, nonce))
mac = cipher_suite.mac(message, key)
print(cipher_suite.check_mac(mac, message, key))
print(cipher_suite.get_enum())

b'hello'
True
CipherSuiteEnum.DH_AES_128_GCM_SHA256
b'hello'
True
CipherSuiteEnum.ECDH_KUZNYECHIK_MGM_STREEBOG


In [3]:
from collections import namedtuple
from pygost.gost3410 import CURVES


curve = CURVES["id-tc26-gost-3410-12-512-paramSetA"]

Certificate = namedtuple('Certificate', ['id', 'public_key', 'signature'])
KeyPair     = namedtuple('KeyPair',     ['public_key', 'private_key'])
Offer       = namedtuple('Offer', ['cipher_suite', 'group', 'generator'])
Mode        = namedtuple('Mode', ['cipher_suite'])

def generate_point():
    temp = gost3410.prv_unmarshal(urandom(64))
    return gost3410.public_key(curve, temp)

def generate_key_pair():
    prv = prv_unmarshal(urandom(64))
    return KeyPair(public_key(curve, prv), prv)

In [4]:
import pickle
from uuid import uuid4
from Crypto.Random import random
from Crypto.Protocol.KDF import HKDF
from Crypto.Util.number import getPrime, getRandomRange, getRandomNBitInteger, long_to_bytes
from pygost.gost3410 import verify
from gostcrypto import gosthmac


# Participant model.
class Participant:
    def __init__(self, ca):
        self._ca = ca
        self._key_pair = generate_key_pair()
#         self._key_pair = KeyPair(self._key_pair[0], self._key_pair[1]+1)  # <------------------------FAULTY LINE--------------------------------
        self._id = uuid4()
        self._certificate = ca.issue_certificate(self)

    # Get current ID.
    def get_id(self):
        return self._id
    
    # Part of zero-knowledge key-statement proof. Returns (Q, P).
    def auth_get_nonce(self):
        self._auth_P = curve.x, curve.y
        self._auth_n = random.getrandbits(64)
        return curve.exp(self._auth_n, *self._auth_P), self._auth_P
    
    # Part of zero-knowledge key-statement proof.
    def auth_get_result(self, c):
        return self._auth_n + c * self._key_pair.private_key
    
    # Get current signature public key.
    def get_public_key(self):
        return self._key_pair.public_key
    
    # Get current certificate.
    def get_certificate(self):
        return self._certificate
    
    # Revoke certificate.
    def revoke_certificate(self, ca):
        ca.revoke(self)
    
    # Sign a message.
    def sign(self, args):
        dgst = gost34112012512.new(args).digest()[::-1]
        signature = sign(curve, self._key_pair.private_key, dgst)
        return signature


# Generic TLS specification participant.
class TlsParticipant(Participant):
    def __init__(self, ca):
        super().__init__(ca)
        
    # Check certificate.
    def check_certificate(self, certificate, id, pk, ca):
        data = str(id).encode() + pub_marshal(pk)
        return ca.check_certificate(certificate) and \
            self.check_signature(certificate.signature, data, ca.get_public_key())
    
    # Check signature.
    def check_signature(self, signature, data, public_key):
        dgst = gost34112012512.new(data).digest()[::-1]
        return verify(curve, public_key, dgst, signature)


# Client model.
class Client(TlsParticipant):
    def __init__(self, ca, cipher_suite: CipherSuite):
        super().__init__(ca)
        self._cipher_suite = cipher_suite
    
    def establish_connection(self, server, connection_type):
        if self._cipher_suite.get_enum() == CipherSuiteEnum.DH_AES_128_GCM_SHA256:
            self.establish_connection_dh(server, connection_type)
        elif self._cipher_suite.get_enum() == CipherSuiteEnum.ECDH_KUZNYECHIK_MGM_STREEBOG:
            self.establish_connection_ecdh(server, connection_type)
    
    # Establish connection using ECDH.
    def establish_connection_ecdh(self, server, connection_type):
        # Generate ECDH parameters.
        p = curve.p
        g = (curve.x, curve.y)
        alpha = getRandomRange(2, p)
        
        # Generate and send ClientHello.
        client_pk = curve.exp(alpha, *g)
        client_nonce = getRandomNBitInteger(2048)
        offer = Offer(self._cipher_suite.get_enum(), p, g)
        
        server_hello = server.tls_get_hello(self, client_pk, client_nonce, offer)
        
        # Process server's response.
        server_pk, server_nonce, mode, c1, c2, c3, c4 = server_hello
        
        # PreSessionKey.
        psk = curve.exp(alpha, *server_pk)
        
        common_bytes = long_to_bytes(client_pk[0]) + long_to_bytes(client_pk[1]) + long_to_bytes(client_nonce) + str(offer).encode() + long_to_bytes(server_pk[0]) + long_to_bytes(server_pk[1]) + long_to_bytes(server_nonce) + str(mode).encode()
        
        # Get Handshake encryption and MAC keys.
        k_ch, k_cm = HKDF(long_to_bytes(psk[0]) + long_to_bytes(psk[1]) + common_bytes,
                          32, b'', SHA256, 2)
        
        print(f'CLIENT: Encryption and MAC keys generated:\n{k_ch = }\n{k_cm = }.')
        
        # Check messages.
        m1 = self._cipher_suite.decrypt(k_ch, *c1)
#         print(m1)

        m2 = pickle.loads(self._cipher_suite.decrypt(k_ch, *c2))
#         print(m2)
        if not self.check_certificate(m2, server.get_id(), server.get_public_key(), self._ca):
            raise ArithmeticError('INVALID CERTIFICATE.')

        m3 = self._cipher_suite.decrypt(k_ch, *c3)
#         print(m3)
        if not self.check_signature(m3, common_bytes + c1[0] + c2[0], server.get_public_key()):
            raise ArithmeticError('INVALID SIGNATURE.')
            
        m4 = self._cipher_suite.decrypt(k_ch, *c4)
#         print(m4)
        if not self._cipher_suite.check_mac(m4, common_bytes + c1[0] + c2[0] + c3[0], k_cm):
            raise ArithmeticError('INVALID MAC.')

        # Construct session keys.
        self._k_cs, self._k_sc = HKDF(long_to_bytes(psk[0]) + long_to_bytes(psk[1]) + common_bytes + c1[0] + c2[0] + c3[0] + c4[0],
                                      32, b'', SHA256, 2)
        
        print(f'CLIENT: Session keys generated:\n{self._k_cs = }\n{self._k_sc = }.')
        
        if m1 == b'yes':
            if not server.check_certificate(self._certificate, self.get_id(), self.get_public_key(), self._ca):
                raise ArithmeticError('INVALID CERTIFICATE.')
    
    # Establish connection using DH.
    def establish_connection_dh(self, server, connection_type):
        # Generate DH parameters.
        p = getPrime(2048)
        g = getRandomRange(2, p)
        alpha = getRandomRange(2, p)
        
        # Generate and send ClientHello.
        client_pk = pow(g, alpha, p)
        client_nonce = getRandomNBitInteger(2048)
        offer = Offer(self._cipher_suite.get_enum(), p, g)
        
        server_hello = server.tls_get_hello(self, client_pk, client_nonce, offer)
        
        # Process server's response.
        server_pk, server_nonce, mode, c1, c2, c3, c4 = server_hello
        
        # PreSessionKey.
        psk = pow(server_pk, alpha, p)
        
        common_bytes = long_to_bytes(client_pk) + long_to_bytes(client_nonce) + str(offer).encode() + long_to_bytes(server_pk) + long_to_bytes(server_nonce) + str(mode).encode()
        
        # Get Handshake encryption and MAC keys.
        k_ch, k_cm = HKDF(long_to_bytes(psk) + common_bytes,
                          32, b'', SHA256, 2)
        
        print(f'CLIENT: Encryption and MAC keys generated:\n{k_ch = }\n{k_cm = }.')
        
        m1 = self._cipher_suite.decrypt(k_ch, *c1)
#         print(m1)

        m2 = pickle.loads(self._cipher_suite.decrypt(k_ch, *c2))
#         print(m2)
        if not self.check_certificate(m2, server.get_id(), server.get_public_key(), self._ca):
            raise ArithmeticError('INVALID CERTIFICATE.')

        m3 = self._cipher_suite.decrypt(k_ch, *c3)
#         print(m3)
        if not self.check_signature(m3, common_bytes + c1[0] + c2[0], server.get_public_key()):
            raise ArithmeticError('INVALID SIGNATURE.')
            
        m4 = self._cipher_suite.decrypt(k_ch, *c4)
#         print(m4)
        if not self._cipher_suite.check_mac(m4, common_bytes + c1[0] + c2[0] + c3[0], k_cm):
            raise ArithmeticError('INVALID MAC.')

        self._k_cs, self._k_sc = HKDF(long_to_bytes(psk) + common_bytes + c1[0] + c2[0] + c3[0] + c4[0],
                                      32, b'', SHA256, 2)
        
        # Construct session keys.
        print(f'CLIENT: Session keys generated:\n{self._k_cs = }\n{self._k_sc = }.')
        
        # If two-way authentication is required, provide certificate to the server.
        if m1 == b'yes':
            if not server.check_certificate(self._certificate, self.get_id(), self.get_public_key(), self._ca):
                raise ArithmeticError('INVALID CERTIFICATE.')

    # Check MAC.
    def check_mac(self, mac, data, key):
        return self._cipher_suite.check_mac(mac, data, key)

    # Send message to the server over an encrypted channel.
    def send_message(self, server, message):
        ct = self._cipher_suite.encrypt(self._k_cs, message)
        server.receive_message(ct)
        
    # Invoked when the message is received.
    def receive_message(self, ct):
        message = self._cipher_suite.decrypt(self._k_sc, *ct)
        print(f'Received on client {message = }.')

    # Update keys.
    def key_update(self):
        self._k_cs = HKDF(self._k_cs, 32, b'', SHA256, 1)
        self._k_sc = HKDF(self._k_sc, 32, b'', SHA256, 1)

# Server model.
class Server(TlsParticipant):
    def __init__(self, ca, cipher_suites: list[CipherSuite]):
        super().__init__(ca)
        self._cipher_suites = cipher_suites
    
    # Check MAC.
    def check_mac(self, mac, data, key):
        return self._cipher_suite.check_mac(mac, data, key)
    
    # DH processor.
    def tls_get_hello_dh(self, client, client_pk, client_nonce, offer):
        # Generate DH parameters.
        _, p, g = offer
        beta = getRandomRange(2, p)
        
        # Generate ServerHello.
        server_pk = pow(g, beta, p)
        server_nonce = getRandomNBitInteger(2048)
        mode = Mode(self._cipher_suite.get_enum())
        
        psk = pow(client_pk, beta, p)
        
        common_bytes = long_to_bytes(client_pk) + long_to_bytes(client_nonce) + str(offer).encode() + long_to_bytes(server_pk) + long_to_bytes(server_nonce) + str(mode).encode()
        
        # Get Handshake encryption and MAC keys.
        k_sh, k_sm = HKDF(long_to_bytes(psk) + common_bytes,
                          32, b'', SHA256, 2)
        
        print(f'SERVER: Encryption and MAC keys generated:\n{k_sh = }\n{k_sm = }.')
        
        cert_request = b'yes'
        c1 = self._cipher_suite.encrypt(k_sh, cert_request)
        c2 = self._cipher_suite.encrypt(k_sh, pickle.dumps(self._certificate))
        c3 = self._cipher_suite.encrypt(k_sh, self.sign(common_bytes + c1[0] + c2[0]))
        c4 = self._cipher_suite.encrypt(k_sh, self._cipher_suite.mac(common_bytes + c1[0] + c2[0] + c3[0], k_sm))

        # Construct session keys.
        self._k_cs, self._k_sc = HKDF(long_to_bytes(psk) + common_bytes + c1[0] + c2[0] + c3[0] + c4[0],
                                      32, b'', SHA256, 2)
        
        print(f'SERVER: Session keys generated:\n{self._k_cs = }\n{self._k_sc = }.')
        
        return server_pk, server_nonce, mode, c1, c2, c3, c4
    
    # Elliptic curve DH processor.
    def tls_get_hello_ecdh(self, client_pk, client_nonce, offer):
        # Generate DH parameters.
        _, p, g = offer
        beta = getRandomRange(2, p)
        
        # Generate ServerHello.
        server_pk = curve.exp(beta, *g)
        server_nonce = getRandomNBitInteger(2048)
        mode = Mode(self._cipher_suite.get_enum())
        
        # PreSessionKey.
        psk = curve.exp(beta, *client_pk)
        
        common_bytes = long_to_bytes(client_pk[0]) + long_to_bytes(client_pk[1]) + long_to_bytes(client_nonce) + str(offer).encode() + long_to_bytes(server_pk[0]) + long_to_bytes(server_pk[1]) + long_to_bytes(server_nonce) + str(mode).encode()
        
        # Get Handshake encryption and MAC keys.
        k_sh, k_sm = HKDF(long_to_bytes(psk[0]) + long_to_bytes(psk[1]) + common_bytes,
                          32, b'', SHA256, 2)
        
        print(f'{k_sh = }\n{k_sm = }.')
        
        # Don't require client certificate.
        cert_request = b'no'
        c1 = self._cipher_suite.encrypt(k_sh, cert_request)
        c2 = self._cipher_suite.encrypt(k_sh, pickle.dumps(self._certificate))
#         c2 = (c2[0], b'wrong bytes', c2[2]) # <------------------------FAULTY LINE--------------------------------
        c3 = self._cipher_suite.encrypt(k_sh, self.sign(common_bytes + c1[0] + c2[0]))
        c4 = self._cipher_suite.encrypt(k_sh, self._cipher_suite.mac(common_bytes + c1[0] + c2[0] + c3[0], k_sm))
        
        self._k_cs, self._k_sc = HKDF(long_to_bytes(psk[0]) + long_to_bytes(psk[1]) + common_bytes + c1[0] + c2[0] + c3[0] + c4[0],
                                      32, b'', SHA256, 2)
        
        # Generate session keys.
        print(f'SERVER: Session keys generated:\n{self._k_cs = }\n{self._k_sc = }.')
        
        return server_pk, server_nonce, mode, c1, c2, c3, c4
    
    # Ask server to return its ServerHello message.
    def tls_get_hello(self, client, client_pk, client_nonce, offer):
        cipher_suite, group, generator = offer

        if cipher_suite == CipherSuiteEnum.DH_AES_128_GCM_SHA256:
            self._cipher_suite = DH_AES_128_GCM_SHA256()
            return self.tls_get_hello_dh(client, client_pk, client_nonce, offer)
        
        if cipher_suite == CipherSuiteEnum.ECDH_KUZNYECHIK_MGM_STREEBOG:
            self._cipher_suite = ECDH_KUZNYECHIK_MGM_STREEBOG()
            return self.tls_get_hello_ecdh(client_pk, client_nonce, offer)
        
    def send_message(self, client, message):
        ct = self._cipher_suite.encrypt(self._k_sc, message)
        client.receive_message(ct)
        
    def receive_message(self, ct):
        message = self._cipher_suite.decrypt(self._k_cs, *ct)
        print(f'Received on server {message = }.')
        
    def key_update(self):
        self._k_cs = HKDF(self._k_cs, 32, b'', SHA256, 1)
        self._k_sc = HKDF(self._k_sc, 32, b'', SHA256, 1)

In [5]:
from pygost import gost34112012512
from pygost.gost3410 import pub_marshal
from pygost.gost3410 import prv_unmarshal
from pygost.gost3410 import public_key
from pygost.gost3410 import sign


# Certificate authority.
class CA:
    def __init__(self):
        self._certificates = set({})
        self._crl = set({})
        self._key_pair = generate_key_pair()

    # Ensure participant has private key corresponding to his public key.
    def check_authenticity(self, participant):
        self._auth_Par_pub = participant.get_public_key()
        self._auth_Q, self._auth_P = participant.auth_get_nonce()
        self._auth_c = random.getrandbits(64)
        self._auth_t = participant.auth_get_result(self._auth_c)
        
        return curve.exp(self._auth_t, *self._auth_P) == \
            curve._add(*self._auth_Q, *curve.exp(self._auth_c, *self._auth_Par_pub))

    # Issue certificate.
    def issue_certificate(self, participant):
        par_id = participant.get_id()
        par_key = participant.get_public_key()
        
        if not self.check_authenticity(participant):
            print('INVALID PRIVATE KEY')
            return

        data_for_signing = str(par_id).encode() + pub_marshal(par_key)
        dgst = gost34112012512.new(data_for_signing).digest()[::-1]
        signature = sign(curve, self._key_pair.private_key, dgst)
        
        # Create certificate and store it.
        certificate = Certificate(par_id, par_key, signature)
        self._certificates.add(certificate)
        
        return certificate
    
    # Revoke certificate.
    def revoke(self, participant):
        if not self.check_authenticity(participant):
            print('INVALID PRIVATE KEY')
            return
        
        par_cert = participant.get_certificate()
        if par_cert in self._certificates:
            self._certificates.remove(par_cert)
            self._crl.add(par_cert)
    
    # Check certificate.
    def check_certificate(self, certificate):
        return (certificate not in self._crl) and (certificate in self._certificates)

    # Get public key.
    def get_public_key(self):
        return self._key_pair.public_key

In [21]:
# Tests

ca = CA()
par = Client(ca, DH_AES_128_GCM_SHA256())
print(f'Certificate check result: {par.check_certificate(par._certificate, par.get_id(), par.get_public_key(), ca)}')
print(f'Registered CA certificates:\n{ca._certificates}\n')
print(f'CRL:\n{ca._crl}\n')

print('--------------------------------------------------------------------------------')

par.revoke_certificate(ca)
print(f'Certificate check result: {par.check_certificate(par._certificate, par.get_id(), par.get_public_key(), ca)}')
print(f'Registered CA certificates:\n{ca._certificates}\n')
print(f'CRL:\n{ca._crl}\n')

print('--------------------------------------------------------------------------------')

par = Client(ca, DH_AES_128_GCM_SHA256())
print(f'Certificate check result: {par.check_certificate(par._certificate, par.get_id(), par.get_public_key(), ca)}')
print(f'Registered CA certificates:\n{ca._certificates}\n')
print(f'CRL:\n{ca._crl}\n')

print('--------------------------------------------------------------------------------')

par.revoke_certificate(ca)
print(f'Certificate check result: {par.check_certificate(par._certificate, par.get_id(), par.get_public_key(), ca)}')
print(f'Registered CA certificates:\n{ca._certificates}\n')
print(f'CRL:\n{ca._crl}\n')

Certificate check result: True
Registered CA certificates:
{Certificate(id=UUID('dfd0853c-07fb-4742-a2bb-0987b5267b10'), public_key=(5542963224921637089601020876799980797354750101292709677315144930227235691438505234274530252716046023223772297541439508070515716730495065353468239547950295, 3118802067576211126459927478930358592168450684373153489352701655336963213238438698149757710735356668571376636406077917723560880499934979079850847473131206), signature=b'\x91\xb2\xee\xca\xabp\x18#:&\x05lE_&\xe5?0\xba\x03\xf4\x95\x90J\x0e\xc9\xed&\xc9,h\x8fU\xa9\x87\x01\x0cZW\x0e7\x90\xc0\xb6\x81\x87\xb9~\x10\x0c<\xc8W\x01\xec\x95\xd5:\xf0\x1ds\x03~\x1dkc\xe1\x0e\x15\x0b\xba\xdf\xa1~\xef\xb2\xd1\xfegtd\xcb72\x956\x14\x8a\xbb4\xd04e\xd2\xfcS\xe9\x18\x9b\x89\x0b<\xaae\xa7^ti\xb7}\xb38\xe5\xc1\x01e\x009\xb3\xe8\x10\x1c\x9az>\xe9P\xaa')}

CRL:
set()

--------------------------------------------------------------------------------
Certificate check result: False
Registered CA certificates:
set()

CRL:
{Certif

In [7]:
ca = CA()
client = Client(ca, ECDH_KUZNYECHIK_MGM_STREEBOG())
server = Server(ca, [DH_AES_128_GCM_SHA256(), ECDH_KUZNYECHIK_MGM_STREEBOG()])

client.establish_connection(server, 1)

message = b'hello'
client.send_message(server, message)
server.send_message(client, message)

client.key_update()
server.key_update()
print(f'Key update happened')

message = b'hello'
client.send_message(server, message)
server.send_message(client, message)

k_sh = b'\xd1\x18\xe2rI\xf5A\xc7n%!\xc6\xa7\x0cG\xb8\xaa\xee}P\xcb\x82g\x84A\x00TG]\x8aZ\xd5'
k_sm = b'\xf5\xdf\xc5V\x157\x83\xc6\x1c\xed\xf7\xed\xab\xaf\xedb\x8a(\x12\xde\xf057\xdcnJw4R\x02$H'
self._k_cs = b'\xbe\x90\xf2a\x8at\x0e.\xc5\x1c\xe2\x8f\x83D\x841~`\x0cO\x12`Q\xaa\xca\xb56~\xf1B\xc0\xeb'
self._k_sc = b"\xb6\xa2\x98\xd3\xbc~\x06\xa3\x16+\xab\xd3'h\xc3C\x7f\xd4\xc8,\x9b\xe7$Z\xf0N\x892\xa5\xfb\xf1\x8c"
k_ch = b'\xd1\x18\xe2rI\xf5A\xc7n%!\xc6\xa7\x0cG\xb8\xaa\xee}P\xcb\x82g\x84A\x00TG]\x8aZ\xd5'
k_cm = b'\xf5\xdf\xc5V\x157\x83\xc6\x1c\xed\xf7\xed\xab\xaf\xedb\x8a(\x12\xde\xf057\xdcnJw4R\x02$H'
self._k_cs = b'\xbe\x90\xf2a\x8at\x0e.\xc5\x1c\xe2\x8f\x83D\x841~`\x0cO\x12`Q\xaa\xca\xb56~\xf1B\xc0\xeb'
self._k_sc = b"\xb6\xa2\x98\xd3\xbc~\x06\xa3\x16+\xab\xd3'h\xc3C\x7f\xd4\xc8,\x9b\xe7$Z\xf0N\x892\xa5\xfb\xf1\x8c"
Received on server message = b'hello'
Received on client message = b'hello'
Key update happened
Received on server message = b'hello'
Received on client message = b'hello

In [8]:
bad_boy = Client(ca, DH_AES_128_GCM_SHA256())
bad_boy._certificate = client._certificate

bad_boy.establish_connection(server, 1)

k_sh = b'\xdc\xa2\xcdb\xc6\x11H\xf3r\xc31\xa8\xc1\x02c\x98\xf2\x1eO\xb15\xcc\xc4r?\x85\x91\xa3\x10"\xfe\x18'
k_sm = b'\xe1\xe8\xff\x14|\x85\x8e)i\x90\x99"\x8e\x11pG\xc8{\x1d\xa8\xbdm@^O\x80\xc9\xafY7\xd5S'
self._k_cs = b'g\xe6{\x98"\x12\xf8\xa7\x8e\xd3\x18\xe5\xfa\xd3\x9e\xf3\xa7\xc5\xa8\x92\x89z\xf8\xe2\xa7\xe2\xddu\xf1s\x80\xc2'
self._k_sc = b'\xbaV\xff\\Y\x91\x1eJ\xe5g8\x0e\x8f"\x88o\xeb\x8e\x86\x9b\x13\xfd\xb4z\x04\x9c\xe0"\xb8\x97\x9f\x8e'
k_ch = b'\xdc\xa2\xcdb\xc6\x11H\xf3r\xc31\xa8\xc1\x02c\x98\xf2\x1eO\xb15\xcc\xc4r?\x85\x91\xa3\x10"\xfe\x18'
k_cm = b'\xe1\xe8\xff\x14|\x85\x8e)i\x90\x99"\x8e\x11pG\xc8{\x1d\xa8\xbdm@^O\x80\xc9\xafY7\xd5S'
self._k_cs = b'g\xe6{\x98"\x12\xf8\xa7\x8e\xd3\x18\xe5\xfa\xd3\x9e\xf3\xa7\xc5\xa8\x92\x89z\xf8\xe2\xa7\xe2\xddu\xf1s\x80\xc2'
self._k_sc = b'\xbaV\xff\\Y\x91\x1eJ\xe5g8\x0e\x8f"\x88o\xeb\x8e\x86\x9b\x13\xfd\xb4z\x04\x9c\xe0"\xb8\x97\x9f\x8e'


ArithmeticError: INVALID CERTIFICATE